In [1]:
import pandas as pd

In [62]:
vitals = pd.read_hdf("all_hourly_data.h5", 'vitals_labs_mean')
print(vitals.shape)

(2200954, 104)


In [3]:
interventions = pd.read_hdf("all_hourly_data.h5",'interventions')
print(interventions.shape)

(2200954, 14)


In [24]:
patients = pd.read_hdf("all_hourly_data.h5", 'patients')
print(patients.shape)

(34472, 28)


In [27]:
# Prepare labels for los > 7 task
patients["los_7"] = patients['los_icu'] > 7
patients.drop(columns=['los_icu'], inplace=True)

In [28]:
patients = patients[["gender","ethnicity","age","los_7","mort_icu"]].reset_index()

In [29]:
patients.los_7 = patients.los_7.apply(int)

In [30]:
print(f"class imbalance for length of stay prediction: {patients.los_7.mean()}")
print(f"class imbalance for icu mortality prediction: {patients.mort_icu.mean()}")

class imbalance for length of stay prediction: 0.053521698770016245
class imbalance for icu mortality prediction: 0.06558946391274077


WINDOW_SIZE determines how much information we want to use for the prediction tasks. For example, a window length of 24 means that we will use the sequence of the first 24 readings for vitals and interventions.

In [31]:
WINDOW_SIZE = 24

In [32]:
interventions = interventions.reset_index()

In [33]:
interventions = interventions[interventions.hours_in < WINDOW_SIZE]

In [40]:
interventions.drop(columns=['level_0','index'], inplace=True)

In [41]:
interventions

,subject_id,hadm_id,icustay_id,hours_in,vent,vaso,adenosine,dobutamine,dopamine,epinephrine,isuprel,milrinone,norepinephrine,phenylephrine,vasopressin,colloid_bolus,crystalloid_bolus,nivdurations
0,3,145834,211552,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,3,145834,211552,1,1,1,0,0,1,0,0,0,0,1,0,0,0,0
2,3,145834,211552,2,1,1,0,0,1,0,0,0,0,1,0,0,0,0
3,3,145834,211552,3,1,1,0,0,0,0,0,0,0,1,0,0,0,0
4,3,145834,211552,4,1,1,0,0,0,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
808534,99999,113369,246512,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0
808535,99999,113369,246512,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0
808536,99999,113369,246512,21,0,0,0,0,0,0,0,0,0,0,0,0,0,0
808537,99999,113369,246512,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [63]:
vitals = vitals.reset_index()

In [64]:
vitals = vitals[vitals.hours_in < WINDOW_SIZE]

In [65]:
vitals.shape

(808539, 108)

In [68]:
vitals.columns = vitals.columns.to_flat_index()
vitals.columns = list(map(lambda x: x[0] + x[1] if x[1] == "" else x[0] + " " + x[1] , vitals.columns.tolist()))

In [69]:
vitals

,subject_id,hadm_id,icustay_id,hours_in,alanine aminotransferase mean,albumin mean,albumin ascites mean,albumin pleural mean,albumin urine mean,alkaline phosphate mean,...,total protein mean,total protein urine mean,troponin-i mean,troponin-t mean,venous pvo2 mean,weight mean,white blood cell count mean,white blood cell count urine mean,ph mean,ph urine mean
0,3,145834,211552,0,25.0,1.8,NaN,NaN,NaN,73.0,...,NaN,NaN,NaN,NaN,NaN,NaN,14.842857,NaN,7.40,5.0
1,3,145834,211552,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,145834,211552,2,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.26,NaN
3,3,145834,211552,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,145834,211552,4,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2200946,99999,113369,246512,19,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2200947,99999,113369,246512,20,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2200948,99999,113369,246512,21,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2200949,99999,113369,246512,22,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
